In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._

import org.apache.spark.rdd.RDD
///////////
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._


import org.apache.spark.sql.functions._


In [2]:
//spark 
val conf: SparkConf = new SparkConf().setMaster("local[*]").setAppName("ww")
val sc: SparkContext = new SparkContext(conf)


conf = org.apache.spark.SparkConf@5fa8d7e6
sc = org.apache.spark.SparkContext@26ba072b


org.apache.spark.SparkContext@26ba072b

In [3]:
//sql
val spark :SparkSession = SparkSession.builder().appName("app").config("spark.some.config.option", "some-value").getOrCreate()
import spark.implicits._ 

spark = org.apache.spark.sql.SparkSession@7a5c24a


org.apache.spark.sql.SparkSession@7a5c24a

In [4]:
val l = List( ("John" , 28 , "new york") ,
              ("Andrew" , 36 , "california") ,
              ("Clarcke" , 22 , "new york") ,
              ("Kevin" , 42 , "denver") ,
              ("Richard" , 51 , "sidney")              )

l = List((John,28,new york), (Andrew,36,california), (Clarcke,22,new york), (Kevin,42,denver), (Richard,51,sidney))


List((John,28,new york), (Andrew,36,california), (Clarcke,22,new york), (Kevin,42,denver), (Richard,51,sidney))

In [8]:
// create new df

//from json using read (csv using read.csv)
// val df: DataFrame = spark.read.json("path")

// from collection 
//l = List ...
// val df  = l.toDF("name", "age" , "city")

// from rdd 
//rdd[(string, int , string)
val rdd1  = sc.parallelize(l) 
// val df  = rdd1.toDF() //df = [_1: string, _2: int ... 1 more field]
val df = rdd1.toDF("name", "age" , "city") // name columns

//from a collection of  class  objects  : column names are automatically set to the attributes of the class 

rdd1 = ParallelCollectionRDD[3] at parallelize at <console>:61
df = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [8]:
//show 
df.show() //show 20 first lines 

// df.show(n)

+-------+---+----------+
|   name|age|      city|
+-------+---+----------+
|   John| 28|  new york|
| Andrew| 36|california|
|Clarcke| 22|  new york|
|  Kevin| 42|    denver|
|Richard| 51|    sidney|
+-------+---+----------+



In [69]:
 df.printSchema // tree

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- city: string (nullable = true)



# RUN SQL PURE QUERIES  , retirn always dfs   :  spark.sql( " sql query ")


https://github.com/apache/spark/blob/master/examples/src/main/scala/org/apache/spark/examples/sql/SparkSQLExample.scala






In [10]:
// name the table  representing the df , its just a way to use select from that table 
df.createOrReplaceTempView("people")  // table is people

val newyork_people = spark.sql("""SELECT * FROM people WHERE city = "new york"  """)  // = or == is the same 
// """ used when we use " "  inside , 
//spark.sql("""SELECT * FROM people WHERE city = "new york" ORDER BY age """)

newyork_people.show

+-------+---+--------+
|   name|age|    city|
+-------+---+--------+
|   John| 28|new york|
|Clarcke| 22|new york|
+-------+---+--------+



newyork_people = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

## SPARK SQL API  :  not sql queries but kind of df.filter ... 

### df. : select , where (filter), limit , orderBy, groupBy, join   

#### 3ways to specify a column :ceg with filter but valable to any operation: grouBy ...

1- using $ :  
df.filter($age > 0)

2- refering to df   
df.filter(df("age") > 0)

3- using sql string   
df.filter("age > 0" )



In [11]:
val smalldf = df.filter($"age" > 0)
smalldf.show

// df.where($"age" > 0).show
// df.filter(df("age") > 0).show
// df.filter("age > 0" ).show


+-------+---+----------+
|   name|age|      city|
+-------+---+----------+
|   John| 28|  new york|
| Andrew| 36|california|
|Clarcke| 22|  new york|
|  Kevin| 42|    denver|
|Richard| 51|    sidney|
+-------+---+----------+



smalldf = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [12]:
// logic

// df.filter($"age" > 0 && $"city" ===  "new york" )   or
df.filter(""" "age" > 0 && "city" = "new york" )   or

Name: Syntax Error.
Message: 
StackTrace: 

In [13]:
df.filter($"city" ===  "new york").show  

// df.filter(df("city") === "new york").show  

// df.filter("""city = "new york" """).show     // or 
// df.filter("city = 'new york' ").show


// =  only with sql like strings 
// values that are strings must always be in ""  , so the sql string must be in """city = "new york" """
//or "city = 'new york' "

+-------+---+--------+
|   name|age|    city|
+-------+---+--------+
|   John| 28|new york|
|Clarcke| 22|new york|
+-------+---+--------+



///////   example   using sql query string to sqy which coluumn 


In [14]:
val newdf = df.select("name" , "age").where("city = 'new york'").orderBy("age") 

// or orderBy(asc/desc("age"))   or  .sort($"col1".desc)
// or orderBy("age" , "name")

newdf.show

+-------+---+
|   name|age|
+-------+---+
|Clarcke| 22|
|   John| 28|
+-------+---+



newdf = [name: string, age: int]


[name: string, age: int]

In [ ]:
val dftest = sc.parallelize(Seq((4, "blah", 2), (2, "", 3), (56, "foo", 3), (100, null, 5)))
    .toDF("A", "B", "C")

val newDf = dftest.withColumn("D", when($"B".isNull or $"B" === "", 0).otherwise(1))
// newDf.show

In [21]:
// df.select("name" , "age").where("city = 'new york'").orderBy("age").show   //saem with
// df.select($"name" , $"age").where("city = 'new york'").orderBy("age").show
df.select(df("name") , $"age").where("city = 'new york'").orderBy("age").show

+-------+---+
|   name|age|
+-------+---+
|Clarcke| 22|
|   John| 28|
+-------+---+



COLLECT

In [15]:
val tomaster = df.collect()
tomaster(0)(0)

tomaster = Array([John,28,new york], [Andrew,36,california], [Clarcke,22,new york], [Kevin,42,denver], [Richard,51,sidney])


John

# groupby  
https://www.coursera.org/learn/scala-spark-big-data/lecture/3U9UW/dataframes-1

groupBdoesnt return a DF  , groupedDataFrame    ,  we then use  -  aggregate : sum($"col"))   
aggregate:  sum   , min , max , avg    .count   or agg(sum or any other , count also )

 or agg(sum()): if we spcify which column to aggregate  (sum,min ... ) on : df.groupBy($"city").agg(min($"age"))   // .orderBy($"count(age)".asc).show 
 
to get different columns with grouBy : df.groupBy("col1" , "col2") and if necessary .count("col1").orderBy($"count(clo1)")

order with count:    orderBy($"count(clo1).asc")



note : df.groupBy($"city",$"age") doesnt do anything since all ages are different: fro groupby must be duplicates for city and also for age : eg:
city0  : age0 (group)
city0  : age1 (other group) 


 #### theres sum count min max ... API on groupeddataset () groupBy
 
 #### and theres agg   API: 
 agg(sum())  : if we  use $"col"    : df.groupBy($"city").agg(min($"age"))   // .orderBy($"count(age)".asc).show 
 #### sum, min , max, avg, mean ,  stddev , first ,last , count  

In [31]:
df.groupBy($"city").count().orderBy($"count".asc).show //df.groupBy("city").

+----------+-----+
|      city|count|
+----------+-----+
|    denver|    1|
|    sidney|    1|
|california|    1|
|  new york|    2|
+----------+-----+



lastException: Throwable = null


In [97]:
// df.groupBy("city").count().orderBy($"count".asc).show
df.groupBy("city").agg(count("*") as "counted").orderBy($"counted".asc).show  // use of agregate y have to specify on which col y want to count or * to say implicit 

+----------+-------+
|      city|counted|
+----------+-------+
|    sidney|      1|
|    denver|      1|
|california|      1|
|  new york|      2|
+----------+-------+



In [12]:
// df.groupBy($"city").agg(count($"age")).orderBy($"count(age)".asc).show  
// ####df.groupBy("city").count("age")     error   use count() 

Name: Compile Error
Message: <console>:53: error: too many arguments for method count: ()org.apache.spark.sql.DataFrame
       df.groupBy("city").count("age")
                               ^

StackTrace: 

In [18]:
// anotherdf.groupBy("zip").max("col")

// df.groupBy($"city").avg("age").show   

df.groupBy($"city").avg("age").orderBy($"avg(age)".asc).show   

+----------+--------+
|      city|avg(age)|
+----------+--------+
|  new york|    25.0|
|california|    36.0|
|    denver|    42.0|
|    sidney|    51.0|
+----------+--------+



scala

// (-2 to -1).map(0.5 + ) is same as  +_

// val f  = (x:Int , y : Char) => {println(x) ; print(y)}
// f(-1,'p')

// val f  = (x:Int) => (y : Char) => {println(x) ; print(y)}
// f(-1)('p') // no f(-1,'p')

// val zscore = (mean:R, sd:R) => (x:R) => (x-mean)/sd

// val v42 = 42

// Some(3) match {
//   case Some(v42) => println("42") //“v42” is interpreted as a name matching any Int value   use `v42`
//   case _ => println("Not 42")
// }

// class C(var x: Char) {
//   def this() { this('u')}
// }

#### missing data   , null or Nan   ,return new dataframe : 

-drop    
-replace 

In [ ]:
df.drop() // drop row if any col in null or NAN  
// drop("all")  dropped only if all cols of the row to drop is null or nan
//dro(Array("col1","col2" ))   drop row only if the cols  , col1-2 are null or nan 

//replace
df.fill(0) //numeric type columns affected 
df.fill(Map("col0" -> val0   , "col1" -> val1 )) // each col is different 
df.replace(Array("col"), Map(val1 , val2)   )  //replace on col , only val1 with val2  only 

## Actions    

collect , cout , first show take 

In [92]:

val l = df.collect()
val u = l(0)   // Array[Row)])
u.getClass

u(0)

l = Array([John,28,new york], [Andrew,36,california], [Clarcke,22,new york], [Kevin,42,denver], [Richard,51,sidney])
u = [John,28,new york]


John

In [93]:
df.head() // .first    return Row


[John,28,new york]

## JOINS

In [ ]:
df1.join(df2  ,   $"df1.col" === $"df2.col") // same col in the two dfs  

// specify outer join
df1.join(df2  ,   $"df1.col" === $"df2.col" , "right_outer") // outer , left_outer , leftsemi


# DATASET  
  /   
  /   
  /   
  /   
  /....    
    
    
      
        









In [14]:
// ************** from DF 

// df.as[(String, Int , String)].show


// from collection of objects  in DF
case class Company(name: String, foundingYear: Int, numEmployees: Int) // following  in another cell , otherwise error retrn .

defined class Company


In [26]:

val inputSeq = Seq(Company("ABC", 1998, 310), Company("XYZ", 1983, 904), Company("NOP", 2005, 83))
val df1 = sc.parallelize(inputSeq).toDF()

val companyDS = df1.as[Company]
companyDS.show


// from rdd: 
// sc.parallelize(inputSeq).toDS().show

//from 
// inputSeq.toDS().show   

+----+------------+------------+
|name|foundingYear|numEmployees|
+----+------------+------------+
| ABC|        1998|         310|
| XYZ|        1983|         904|
| NOP|        2005|          83|
+----+------------+------------+



inputSeq = List(Company(ABC,1998,310), Company(XYZ,1983,904), Company(NOP,2005,83))
df1 = [name: string, foundingYear: int ... 1 more field]
companyDS = [name: string, foundingYear: int ... 1 more field]


[name: string, foundingYear: int ... 1 more field]

In [6]:
// // from collection 

l.toDS()  // default cols 

val cols = List("name", "age" , "city") 
// toDF(colsits incorrect because toDf("name", "age" , "city") but not toDF( ("name", "age" , "city")  )   we use :  varargs passing 


val ds = l.toDF(cols:_*).as[(String,Int,String)]     
ds.show


// 

// //from rdd
// rdd1.toDS().show

+-------+---+----------+
|   name|age|      city|
+-------+---+----------+
|   John| 28|  new york|
| Andrew| 36|california|
|Clarcke| 22|  new york|
|  Kevin| 42|    denver|
|Richard| 51|    sidney|
+-------+---+----------+



cols = List(name, age, city)
ds = [name: string, age: int ... 1 more field]


[name: string, age: int ... 1 more field]

In [105]:
val wordsDataset = sc.parallelize(Seq("Spark I am your father"     ,       "May the spark be with you"   ,      "Spark I am your father")).toDS()
// wordsDataset.show

val groupedDataset = wordsDataset.flatMap(_.toLowerCase.split(" "))
                                 .filter(_ != "")
                                 .groupBy("value") //the columns is called value by default 


val countsDataset = groupedDataset.count()
countsDataset.show()
// countsDataset.groupBy("count").agg(count("value")).show


+------+-----+
| value|count|
+------+-----+
|father|    2|
|   you|    1|
|  with|    1|
|    be|    1|
|  your|    2|
|   may|    1|
| spark|    3|
|   the|    1|
|     i|    2|
|    am|    2|
+------+-----+



wordsDataset = [value: string]
groupedDataset = RelationalGroupedDataset: [grouping expressions: [value: string], value: [value: string], type: GroupBy]
countsDataset = [value: string, count: bigint]


[value: string, count: bigint]

## combining    Functional  rdd  and Relational sql   operations on DS

In [50]:
import org.apache.spark.sql.functions._

val wordsDataset = sc.parallelize(Seq("Spark I am your father", "May the spark be with you", "Spark I am your father")).toDS()
val result = wordsDataset
              .flatMap(_.split(" "))               // Split on whitespace
              .filter(_ != "")                     // Filter empty words
              .map(_.toLowerCase())
              .groupBy($"value")                   // Count number of occurrences of each word
              .agg(count("*") as "numOccurances")
              .orderBy($"numOccurances" desc)      // Show most common words first
result.show()    //DS 




// agg(avg($"col".as[]Double))




+------+-------------+
| value|numOccurances|
+------+-------------+
| spark|            3|
|  your|            2|
|father|            2|
|     i|            2|
|    am|            2|
|  with|            1|
|    be|            1|
|   you|            1|
|   the|            1|
|   may|            1|
+------+-------------+



wordsDataset = [value: string]
result = [value: string, numOccurances: bigint]


[value: string, numOccurances: bigint]

y might  use functional  on DF : df.map( row => row(0).asInstanceOf[type] )   it return a dataset : type specification required asInstance Of


In [10]:
// DF 
// df.map( row => row(0).asInstanceOf[String]).show

//DS
ds.map( tuple => (tuple._1 , tuple._2)).show

+-------+---+
|     _1| _2|
+-------+---+
|   John| 28|
| Andrew| 36|
|Clarcke| 22|
|  Kevin| 42|
|Richard| 51|
+-------+---+



In [ ]:
ds.filter($"city".as[String] === "thecity")

In [86]:
ds.select($"name" , $"city").show
// ds.distinct().show

+-------+----------+
|   name|      city|
+-------+----------+
|   John|  new york|
| Andrew|california|
|Clarcke|  new york|
|  Kevin|    denver|
|Richard|    sidney|
+-------+----------+



# DF has rows as single element (row(0) ... )  : NOT TYPED ,   DATASET has  TUPLES :  eg  ds.map(x  =>   )    x is a tuple representing elements of the same row  : TYPED ,careful or y get errors , but in DF the compiler dont detect type problems ,like sql 


In [93]:
ds.map(x => x._1 + " , age " + x._2.toString ).show  //x is a tuple 

+----------------+
|           value|
+----------------+
|   John , age 28|
| Andrew , age 36|
|Clarcke , age 22|
|  Kevin , age 42|
|Richard , age 51|
+----------------+



# groupby key

groupby key (l => l._1)    // l is a tuple (String, Int, String) because its DS not DF 
the result: keyvaluegroupeddataset     must be followed by affregation  

after you can use :   
agg (col : V) : DS{K,U}  
reducegroups (f: (V V ) =>V) returns a DS(K,V)  , reduces elements grouped using f functin like reduce acc: V , cur : V return V  
mapGroup( f(K, Iterator[]V => U   )  )   : Dataset[]U
mapValues
//  
//  
even using grouby KEY : the groups still contain all the columns including key 

In [121]:
val dsgrouped = ds.groupByKey(l => l._1)  // l is a tuple because its DS not DF 
dsgrouped.agg(avg("age").as[Double]).show   
//ds.groupByKey(l => l._1).agg(avg("age")).show    ERROR      use agg(avg("age").as[Double] to avoiid 

+-------+--------+
|  value|avg(age)|
+-------+--------+
|Clarcke|    22.0|
|   John|    28.0|
|Richard|    51.0|
| Andrew|    36.0|
|  Kevin|    42.0|
+-------+--------+



dsgrouped = KeyValueGroupedDataset: [key: [value: string], value: [_1: string, _2: int ... 1 more field(s)]]


KeyValueGroupedDataset: [key: [value: string], value: [_1: string, _2: int ... 1 more field(s)]]

In [15]:
ds.groupByKey(l => (l._1,l._3)).agg(avg("age").as[Double]).show        // l is a tuple because its DS not DF 

Name: Compile Error
Message: <console>:62: error: type mismatch;
 found   : String("avg")
 required: org.apache.spark.sql.Encoder[?]
       ds.groupByKey(l => (l._1,l._3)).agg(avg("age").as[Double] as "avg").show        // l is a tuple because its DS not DF
                                                                    ^

StackTrace: 

In [131]:
//reduceGroups(f: (V V ) =>V) returns a DS(K,V)   : V is the tuple (String, Int, String)
// val dsgrouped = ds.groupByKey(l => l._1)

dsgrouped.reduceGroups((acc , y ) => (y._1, y._2 ,y._3) ).show // last tuple for each group     
// (f: ((String, Int, String), (String, Int, String)) => (String, Int, String))


+-------+------------------------------+
|  value|ReduceAggregator(scala.Tuple3)|
+-------+------------------------------+
|Clarcke|          [Clarcke, 22, new...|
|   John|          [John, 28, new york]|
|Richard|          [Richard, 51, sid...|
| Andrew|          [Andrew, 36, cali...|
|  Kevin|           [Kevin, 42, denver]|
+-------+------------------------------+



In [174]:
// dsgrouped.keys.show
dsgrouped.mapGroups((K , group) => { val tupl : (String,Int,String) = group.next ;  (tupl._1) }   ).show   //group iterator so next only 

//group contain the key  also , 

+-------+
|  value|
+-------+
|Clarcke|
|   John|
|Richard|
| Andrew|
|  Kevin|
+-------+



In [13]:
val dstest  = List( (-1,"kl") , (-1,"ku") , (-1,"kp") , (-2,"jiu") , (-2,"gjhj")    ).toDS()
// dstest.show 
//dstest.groupByKey(tupl => tupl._1).reduceGroups( (acc : (Int,String), c:(Int,String) ) =>    (acc._1 + c._1 , acc._2 + c._2)    ) // same type 

// the performance isnt good because foldleft is not parrallel at all 
// dstest.groupByKey(tupl => tupl._1).mapGroups((k , group) => (k , group.foldLeft("")((acc :String ,p : (Int,String)) => acc + p._2)) ).orderBy($"_1").show


//using reduceGroups 
dstest.groupByKey(p => p._1)
        .mapValues(p => p._2) // delete the key in the values 
          .reduceGroups((acc : String , str) => acc + str )      // (v,v) => v
           .show


// not performant 

+---+-------+
| _1|     _2|
+---+-------+
| -2|jiugjhj|
| -1| klkukp|
+---+-------+

+-----+----------------------------------+
|value|ReduceAggregator(java.lang.String)|
+-----+----------------------------------+
|   -1|                            klkukp|
|   -2|                           jiugjhj|
+-----+----------------------------------+



dstest = [_1: int, _2: string]


[_1: int, _2: string]

# Aggregator  : like the rdd aggregator ( ( foldleft ) then  fold )  //parallelizable
class aggregator [IN , BUF , OUT]


In [21]:
import org.apache.spark.sql.expressions.Aggregator
import org.apache.spark.sql.Encoder


val dstest  = List( (-1,"kl") , (-1,"ku") , (-1,"kp") , (-2,"jiu") , (-2,"gjhj")    ).toDS()
// how to concat all the string by key

dstest = [_1: int, _2: string]


[_1: int, _2: string]

In [33]:


val strConcat = new Aggregator[(Int,String) , String , String]{   // [ IN ,BUF , OUT   )  ====> (Int,String) , String , String
    import org.apache.spark.sql.Encoders
    
    def zero : String  = ""
    def reduce(b:String , a:(Int,String) ): String  = a._2 + b
    def merge(b1: String ,  b2: String ): String = b1 + b2
    def finish(r:String ):  String  =     r
    override def bufferEncoder: Encoder[String] =  Encoders.STRING //    override def bufferEncoder: Encoder[BUF] = 
    override def outputEncoder: Encoder[String] =   Encoders.STRING//   override def bufferEncoder: Encoder[OUT] = 

    
}.toColumn()


dstest.groupByKey(p => p._1)
          .agg(strConcat.as[String])

Name: java.lang.RuntimeException
Message: Unsupported literal type class scala.runtime.BoxedUnit ()
StackTrace: java.lang.RuntimeException: Unsupported literal type class scala.runtime.BoxedUnit ()
  at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:77)
  at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:163)
  at org.apache.spark.sql.catalyst.expressions.Literal$$anonfun$create$2.apply(literals.scala:163)
  at scala.util.Try.getOrElse(Try.scala:79)
  at org.apache.spark.sql.catalyst.expressions.Literal$.create(literals.scala:162)
  at org.apache.spark.sql.functions$.typedLit(functions.scala:113)
  at org.apache.spark.sql.functions$.lit(functions.scala:96)
  at org.apache.spark.sql.Column.apply(Column.scala:212)

In [34]:
//Actions on DS
val k = ds.collect() 
k(0)._1
//cand do anything on the tuple 


k = Array((John,28,new york), (Andrew,36,california), (Clarcke,22,new york), (Kevin,42,denver), (Richard,51,sidney))


lastException: Throwable = null


John

//on DS: collect , count , first , foreach , reduce , show , take 

# When to use DF DS RDD
DS: 
- y work on structured/semi structered data
-y want typesafety
-y need to use FUNCTIONAL API
-y need good performance, but not necessary the best performanc ,

DF: 
- y work on structured/semi structered data
- you want the best performance , optimized for you

RDD: 
-y have unstrcutered data 
- y need to fine tune and manage low-level details od rdd computations
-you have data types that cannot be serialized by the spark sql Encoder (not supported)
